# Step 1: Program requirements
### Import dependencies, load and set environment variables

In [12]:
# Import dependencies
import subprocess
import json
import os

from dotenv import load_dotenv
import import_ipynb
import constants

from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import wif_to_key

from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware

from pprint import pprint

# Load and set environment variables

# Import necessary functions from bit and web3
w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

load_dotenv('../blockchain_hw19.env')
mnemonic=os.getenv("mnemonic")

# set gas price strategy to built-in "medium" algorithm (est ~5min per tx)
# see https://web3py.readthedocs.io/en/stable/gas_price.html?highlight=gas
# see https://ethgasstation.info/ API for a more accurate strategy
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)


# Step 2: Key functions
### 1) derive_wallets: Lookup and return account keys based on 12 word mnemonic
### 2) priv_key_to_account: Based on the coin passed to the function, looks up the coin's address and private key stored in the coin dictionary.
### 3) create_txn: Creates an unsigned transaction appropriate metadata.
### 4) send_txn: Calls create_txn, signs and sends the transaction.

In [13]:
# Create a function called `derive_wallets`
def derive_wallets(coin, mnemonic=mnemonic, depth=3):
    wallet_fpath = '/Users/rhnil/Desktop/Columbia_Bootcamp/Blockchain_Python_homework19/wallet/hd-wallet-derive/hd-wallet-derive.php'
    command = f'php {wallet_fpath} -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [14]:
# Create a dictionary object called coins to store the output from `derive_wallets`.

coins = {'btc': derive_wallets(constants.BTC),
         'eth': derive_wallets(constants.ETH),
         'btc-test': derive_wallets(constants.BTCTEST), 
}

#pprint(coins)

In [15]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin):
    if coin == 'eth':
        list_of_accounts = coins.get('eth')
        account_info =[]
        for coin in list_of_accounts:
            account_info.append(tuple((coin['address'],coin['privkey'])))
        return account_info
            
    if coin == 'btc-test':
        list_of_accounts = coins.get('btc-test')
        account_info =[]
        for coin in list_of_accounts:
            account_info.append(tuple((coin['address'], coin['privkey'])))
        return account_info
       


# Create a function called `create_txn` that creates an unsigned transaction appropriate metadata.
def create_txn(coin, account, recipient, amount):
    if coin == 'eth':
        value = w3.toWei(amount, "ether")
        gasEstimate = w3.eth.estimateGas({"to":recipient, "from":account, "amount":amount})
        return {
            "to":recipient,
            "from":account,
            "value":value,
            "gas":gasEstimate,
            "gasPrice":w3.eth.generateGasPrice(),
            "nonce":w3.eth.getTransactionCount(account),
            "chainId":w3.eth.chain_id
        }
    if coin == 'btc-test':
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, constants.BTC)])

# Create a function called `send_txn` that calls `create_txn`, signs and sends the transaction.
def send_txn(coin, account, recipient, amount):
    if coin == 'eth':
        raw_txn = create_txn(coin, account.address, recipient, amount)
        signed_txn = account.signTransaction(raw_txn)
        return w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    if coin == 'btc-test':
        raw_txn = create_txn(coin, account, recipient, amount)
        print(raw_txn)
        signed_txn = account.sign_transaction(raw_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

# Step 3: Test functions
### 1) Send ETH transaction and confirm via updated ETH balance
### 2) Send BTC-TEST transaction and confirm via updated BTC-TEST balance

In [48]:
# Get current eth account address and eth balance
sender_account_address = priv_key_to_account('eth')[0][0]
sender_eth_balance = w3.fromWei(w3.eth.getBalance(sender_account_address), "ether")
sender_key = Account.privateKeyToAccount(priv_key_to_account('eth')[0][1])

receiver_account_address = priv_key_to_account('eth')[1][0]
receiver_eth_balance = w3.fromWei(w3.eth.getBalance(receiver_account_address), "ether")

print(f"Sender account address: {sender_account_address}")
print(f"Sender ETH balance: {sender_eth_balance}")

print(f"Receiver account address: {receiver_account_address}")
print(f"Receiver ETH balance: {receiver_eth_balance}")

Sender account address: 0xf1BC2ABd37fC0A4B05C81b1d2c9a3ce90139BE63
Sender ETH balance: 10.84344156
Receiver account address: 0xC89044d5dC88633c39c1647C816462d4e55a10E5
Receiver ETH balance: 57.97819024


In [49]:
# Send eth 
send_txn('eth', sender_key, receiver_account_address, 1)

account.address: 0xf1BC2ABd37fC0A4B05C81b1d2c9a3ce90139BE63
type account.address:<class 'str'>
raw_txn: {'to': '0xC89044d5dC88633c39c1647C816462d4e55a10E5', 'from': '0xf1BC2ABd37fC0A4B05C81b1d2c9a3ce90139BE63', 'value': 1000000000000000000, 'gas': 21000, 'gasPrice': 20000000000, 'nonce': 49, 'chainId': 1337}
type raw_txn: <class 'dict'>


HexBytes('0x6a4c7e88505d0918b6c02b4737f13251a1c8c2fe730025d0c89414dcf8d4c843')

In [50]:
# Get updated eth account address and eth balance
sender_account_address = priv_key_to_account('eth')[0][0]
sender_eth_balance = w3.fromWei(w3.eth.getBalance(sender_account_address), "ether")

receiver_account_address = priv_key_to_account('eth')[1][0]
receiver_eth_balance = w3.fromWei(w3.eth.getBalance(receiver_account_address), "ether")


print(f"Sender account address: {sender_account_address}")
print(f"Sender ETH balance: {sender_eth_balance}")

print(f"Receiver account address: {receiver_account_address}")
print(f"Receiver ETH balance: {receiver_eth_balance}")

Sender account address: 0xf1BC2ABd37fC0A4B05C81b1d2c9a3ce90139BE63
Sender ETH balance: 9.84302156
Receiver account address: 0xC89044d5dC88633c39c1647C816462d4e55a10E5
Receiver ETH balance: 58.97819024


In [21]:
# Get account address and current btc-test balance
account_address = priv_key_to_account('btc-test')[0][0]
key = wif_to_key(priv_key_to_account('btc-test')[0][1])
print(f"Account address: {account_address}")
print(f"BTC-TEST balance: {key.get_balance('btc')}")

Account address: n1DD8g8Y7h7scEaSq7wHoG4CNR3DphKuoi
BTC-TEST balance: 0.00100652


In [23]:
# Send btc-test and get new account balance
send_txn('btc-test', key, account_address, 0.000001)
print(f"Account address: {account_address}")
print(f"BTC-TEST balance: {key.get_balance('btc')}")

{"unspents":[{"amount":100,"confirmations":0,"script":"76a914d8076a2d42ce4cd0230bfb84e02ab87f88b35e8688ac","txid":"151f196f99c26c1d06bdf2c1815bf027ca357e08421191c711e29b77a152325d","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":47308,"confirmations":0,"script":"76a914d8076a2d42ce4cd0230bfb84e02ab87f88b35e8688ac","txid":"151f196f99c26c1d06bdf2c1815bf027ca357e08421191c711e29b77a152325d","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["n1DD8g8Y7h7scEaSq7wHoG4CNR3DphKuoi",100],["n1DD8g8Y7h7scEaSq7wHoG4CNR3DphKuoi",9160]]}
Account address: n1DD8g8Y7h7scEaSq7wHoG4CNR3DphKuoi
BTC-TEST balance: 0.00047408
